In [1]:
import openai
from docx import Document
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import os
import json

In [2]:
# Установите ваш OpenAI API ключ
openai.api_key = ""

In [3]:
# Укажите путь к файлу / chemin d'accès au fichier
downloads_path = os.path.expanduser("Desktop")  # Путь к папке "Загрузки"
file_name = "Bat-ADAPT.docx"  # Название файла
file_path = os.path.join(downloads_path, file_name)

In [4]:
# Извлечение текста из Word
# Extraire le texte de Word
def extract_text_from_word(file_path):
    doc = Document(file_path)
    text = "\n".join([p.text for p in doc.paragraphs if p.text.strip()])
    return text

In [5]:
document_text = extract_text_from_word(file_path)
print("Contenu du document:")
print(document_text[:500])  # Выводим первые 500 символов / Nous affichons les 500 premiers caractères

Contenu du document:
Exemples de Questions et Réponses pouvant alimenter le Chatbox de Bat-ADAPT
Périmètre géographique des analyses Bat-ADAPT 
Question : Dans quels pays en Europe peut-on utiliser cet outil ? Est-ce possible au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque ?
Réponse : L'outil Bat-ADAPT permet des analyses de risques climatiques des bâtiments situés en Europe Continentale. Il est tout à fait possible d'utiliser Bat-ADAPT pour les bâtiments situés au Royaume-Uni, en Suisse, en Pologne o


In [6]:
# Добавьте вывод всех параграфов документа, чтобы увидеть, как python-docx извлекает текст:
# Ajouter une impression de tous les paragraphes du document pour voir comment python-docx extrait le texte :

def debug_document(file_path):
    doc = Document(file_path)
    print("Contenu du document:")
    for i, paragraph in enumerate(doc.paragraphs):
        print(f"Paragraphe {i + 1}: '{paragraph.text}'")

# Путь к вашему Word-файлу
debug_document(file_path)

Contenu du document:
Paragraphe 1: 'Exemples de Questions et Réponses pouvant alimenter le Chatbox de Bat-ADAPT'
Paragraphe 2: ''
Paragraphe 3: 'Périmètre géographique des analyses Bat-ADAPT '
Paragraphe 4: ''
Paragraphe 5: 'Question : Dans quels pays en Europe peut-on utiliser cet outil ? Est-ce possible au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque ?'
Paragraphe 6: ''
Paragraphe 7: 'Réponse : L'outil Bat-ADAPT permet des analyses de risques climatiques des bâtiments situés en Europe Continentale. Il est tout à fait possible d'utiliser Bat-ADAPT pour les bâtiments situés au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque. '
Paragraphe 8: ''
Paragraphe 9: 'Gestion de la base de données de bâtiments '
Paragraphe 10: ''
Paragraphe 11: 'Question : Quand nous ajoutons un nouveau fichier Excel dans notre espace "Mon Patrimoine" alors qu’un précédent Excel avait été importé, est-ce que cela va écraser les précédentes données ?'
Paragraphe 12: ''
Paragraphe 13: 'R

In [7]:
# Извлечение текста из документа Word
# Extraire du texte d'un document Word

def extract_questions_answers(file_path):
    """
    Читает вопросы и ответы из файла .docx.
    Возвращает список пар (вопрос, ответ).

    Lit les questions et les réponses à partir d'un fichier .docx.
    Renvoie une liste de paires (question, réponse).
    
    """
    doc = Document(file_path)
    qa_pairs = []
    current_question = None

    # Проходим по каждому абзацу
    # Nous parcourons chaque paragraphe
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        
        # Убираем неразрывные пробелы и приводим к стандартному виду
        # Supprimer les espaces insécables et ramener au format standard
        text = text.replace('\xa0', ' ').strip()

        # Проверяем на вопрос
        # Vérifiez la question
        if text.lower().startswith("question :"):
            current_question = text.replace("Question :", "").strip()
        
        # Проверяем на ответ
        # Vérifier la réponse
        elif text.lower().startswith("réponse :") and current_question:
            answer = text.replace("Réponse :", "").strip()
            qa_pairs.append({"question": current_question, "answer": answer})
            current_question = None
    
    return qa_pairs

In [8]:
qa_pairs = extract_questions_answers(file_path)
if not qa_pairs:
    print("Erreur : Aucune question ou réponse trouvée dans le document.")
else:
    print(f"{len(qa_pairs)} paires de questions et réponses trouvées:")
    for pair in qa_pairs:
        print(f"Question: {pair['question']}")
        print(f"Réponses: {pair['answer']}\n")

9 paires de questions et réponses trouvées:
Question: Dans quels pays en Europe peut-on utiliser cet outil ? Est-ce possible au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque ?
Réponses: L'outil Bat-ADAPT permet des analyses de risques climatiques des bâtiments situés en Europe Continentale. Il est tout à fait possible d'utiliser Bat-ADAPT pour les bâtiments situés au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque.

Question: Quand nous ajoutons un nouveau fichier Excel dans notre espace "Mon Patrimoine" alors qu’un précédent Excel avait été importé, est-ce que cela va écraser les précédentes données ?
Réponses: Oui, les données des bâtiments ayant le même "Code de bâtiment" seront écrasées par celles du nouveau fichier Excel importé.

Question: Comment peut-on supprimer un bâtiment, notamment les bâtiments qui ont servi de test ?
Réponses: Pour supprimer des bâtiments, il faut les sélectionner dans la liste en cochant leurs cases.

Question: Est-il possible d

In [9]:
# Разбиение текста на блоки
# Diviser le texte en blocs
def split_text_into_chunks(text, max_tokens=1500):
    sentences = text.split("\n")
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        token_length = len(sentence.split())
        if current_length + token_length > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(sentence)
        current_length += token_length

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

In [10]:
document_text_2 = split_text_into_chunks(document_text)
print("Contenu du document:")
print(document_text_2[:2000])  # Выводим первые 2000 символов

Contenu du document:
['Exemples de Questions et Réponses pouvant alimenter le Chatbox de Bat-ADAPT Périmètre géographique des analyses Bat-ADAPT  Question\xa0: Dans quels pays en Europe peut-on utiliser cet outil ? Est-ce possible au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque ? Réponse\xa0: L\'outil Bat-ADAPT permet des analyses de risques climatiques des bâtiments situés en Europe Continentale. Il est tout à fait possible d\'utiliser Bat-ADAPT pour les bâtiments situés au\xa0Royaume-Uni, en Suisse, en Pologne ou en République Tchèque.\xa0 Gestion de la base de données de bâtiments  Question\xa0: Quand nous ajoutons un nouveau fichier Excel dans notre espace "Mon Patrimoine" alors qu’un précédent Excel avait été importé, est-ce que cela va écraser les précédentes données ? Réponse\xa0: Oui, les données des bâtiments ayant le même "Code de bâtiment" seront écrasées par celles du nouveau fichier Excel importé. Question\xa0: Comment peut-on supprimer un bâtiment, notammen

In [11]:
# Fix truncated JSON
def fix_truncated_json(json_string):
    try:
        return json.loads(json_string)
    except json.JSONDecodeError:
        # Attempt to fix truncated JSON
        if json_string.strip()[-1] not in [']', '}']:
            json_string += ']'
        try:
            return json.loads(json_string)
        except json.JSONDecodeError:
            print("Failed to fix truncated JSON.")
            return []

In [12]:
# Create a vector index
def create_vector_index(chunks, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(chunks, convert_to_tensor=False)
    dimension = embeddings[0].shape[0]

    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return index, model, embeddings

In [13]:
# Search relevant chunks
def search_relevant_chunks(query, index, model, chunks, top_k=3):
    query_embedding = model.encode([query], convert_to_tensor=False)
    distances, indices = index.search(np.array(query_embedding), top_k)
    results = [{"chunk": chunks[idx], "distance": dist} for dist, idx in zip(distances[0], indices[0])]
    return results

In [14]:
# Extract questions and answers using LLM
def extract_qa_with_llm_safe(text_chunk):
    chunks = split_text_into_chunks(text_chunk)
    all_qa_pairs = []

    for chunk in chunks:
        prompt = (
            f"Please extract only the questions and answers from the text below. "
            f"Ensure the output is a valid JSON array in the following format:\n"
            f"[{{question': 'Question text', 'answer': 'Answer text'}}]\n"
            f"Do not include any additional text or explanations. Only output the JSON array.\n"
            f"Text:\n{chunk}"
        )
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1000,
            temperature=0
        )
        llm_output = response.choices[0].message.content.strip()
        try:
            # Validate and parse the JSON response
            qa_pairs = fix_truncated_json(llm_output)
            all_qa_pairs.extend(qa_pairs)
        except Exception as e:
            print(f"Error processing chunk: {e}")
            print(f"LLM output: {llm_output}")
    return all_qa_pairs

In [15]:
extract_qa_with_llm_safe(document_text)

[{'question': 'Dans quels pays en Europe peut-on utiliser cet outil ? Est-ce possible au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque ?',
  'answer': "L'outil Bat-ADAPT permet des analyses de risques climatiques des bâtiments situés en Europe Continentale. Il est tout à fait possible d'utiliser Bat-ADAPT pour les bâtiments situés au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque."},
 {'question': 'Quand nous ajoutons un nouveau fichier Excel dans notre espace "Mon Patrimoine" alors qu’un précédent Excel avait été importé, est-ce que cela va écraser les précédentes données ?',
  'answer': 'Oui, les données des bâtiments ayant le même "Code de bâtiment" seront écrasées par celles du nouveau fichier Excel importé.'},
 {'question': 'Comment peut-on supprimer un bâtiment, notamment les bâtiments qui ont servi de test ?',
  'answer': 'Pour supprimer des bâtiments, il faut les sélectionner dans la liste en cochant leurs cases.'},
 {'question': 'Est-il possible de m

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

def find_best_match(query, qa_pairs, model):
    """
    Находит наиболее релевантный вопрос-ответ на основе сходства с запросом.
    Trouve la question-réponse la plus pertinente en fonction de la similitude avec la requête.
    """
    questions = [qa['question'] for qa in qa_pairs]
    question_embeddings = model.encode(questions, convert_to_tensor=False)
    query_embedding = model.encode([query], convert_to_tensor=False)

    # Вычисляем сходство косинуса
    similarities = cosine_similarity(query_embedding, question_embeddings).flatten()
    best_index = similarities.argmax()

    return qa_pairs[best_index]

In [17]:
def process_with_vectorization_and_llm(file_path, query):
    # Step 1: Extract text
    text = extract_text_from_word(file_path)
    chunks = split_text_into_chunks(text)

    # Step 2: Create vector index
    index, model, embeddings = create_vector_index(chunks)

    # Step 3: Search relevant chunks
    relevant_chunks = search_relevant_chunks(query, index, model, chunks, top_k=3)

    # Step 4: Extract Q&A pairs with LLM
    qa_pairs = []
    for chunk_data in relevant_chunks:
        chunk = chunk_data["chunk"]
        try:
            llm_response = extract_qa_with_llm_safe(chunk)
            qa_pairs.extend(llm_response)
        except Exception as e:
            print(f"Error processing chunk: {e}")

    # Step 5: Find the best match for the query
    if qa_pairs:
        best_match = find_best_match(query, qa_pairs, model)
        return best_match
    else:
        return None

In [18]:
query = "Comment peut-on supprimer un bâtiment, notamment les bâtiments qui ont servi de test ?"

In [19]:
process_with_vectorization_and_llm(file_path, query)

Failed to fix truncated JSON.


{'question': 'Comment peut-on supprimer un bâtiment, notamment les bâtiments qui ont servi de test ?',
 'answer': 'Pour supprimer des bâtiments, il faut les sélectionner dans la liste en cochant leurs cases.'}

In [20]:
def main():
    downloads_path = os.path.expanduser("Desktop")  # Путь к папке "Загрузки"
    file_name = "Bat-ADAPT.docx"  # Название файла
    file_path = os.path.join(downloads_path, file_name)

    print("Chatbot est prêt ! Posez votre question :")
    while True:
        user_query = input("Vous : ")
        if user_query.lower() in ["exit", "quit"]:
            print("Chatbot a terminé. Au revoir !")
            break
        best_answer = process_with_vectorization_and_llm(file_path, user_query)

        if best_answer:
            print("Extracted questions and answers:")
            print(f"Question: {best_answer['question']}")
            print(f"Answer: {best_answer['answer']}")
        else:
            print("No relevant answer found.")
       
        

if __name__ == "__main__":
    main()

Chatbot est prêt ! Posez votre question :


Vous :  Comment peut-on supprimer un bâtiment, notamment les bâtiments qui ont servi de test ?


Failed to fix truncated JSON.
Extracted questions and answers:
Question: Comment peut-on supprimer un bâtiment, notamment les bâtiments qui ont servi de test ?
Answer: Pour supprimer des bâtiments, il faut les sélectionner dans la liste en cochant leurs cases.


Vous :  Dans quels pays en Europe peut-on utiliser cet outil ? Est-ce possible au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque ?


Extracted questions and answers:
Question: Dans quels pays en Europe peut-on utiliser cet outil ? Est-ce possible au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque ?
Answer: L'outil Bat-ADAPT permet des analyses de risques climatiques des bâtiments situés en Europe Continentale. Il est tout à fait possible d'utiliser Bat-ADAPT pour les bâtiments situés au Royaume-Uni, en Suisse, en Pologne ou en République Tchèque.


Vous :  exit


Chatbot a terminé. Au revoir !
